## 207 -Applied Machine Learning Project: Predicting Attrition of an Online Store Site

#### Authors:

Diego Moss\
Sammy Cayo\
Conor Huh\
Roz Huang\
Jasmine Lau



## Loading Data from BigQuery and Binding Rows

In [1]:
## code block for initial data loading

from google.colab import auth
import numpy as np
import pandas as pd
from google.cloud import bigquery
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import sys


def query_data():
  sys.path.append('/content/src/bq-helper')
  from bq_helper import BigQueryHelper
  !pip install -e git+https://github.com/ConorHuh/BigQuery_Helper#egg=bq_helper
  auth.authenticate_user()
  project_id = "final-project-430217" # change this to correct project ID
  client = bigquery.Client(project=project_id)

  google_analytics = BigQueryHelper(active_project="bigquery-public-data",
                                    dataset_name="data:google_analytics_sample",
                                    project_id=project_id)

  table_names = []
  all_data = []
  all_hits_data = []

  start_date = datetime(2016, 8, 1)
  end_date = datetime(2017, 8, 1)
  current_date = start_date

  while current_date <= end_date:
      table_name = f"ga_sessions_{current_date.strftime('%Y%m%d')}"

      query = f"""
      SELECT *
      FROM `bigquery-public-data.google_analytics_sample.{table_name}`
      """
      table_names.append(table_name)
      df = google_analytics.query_to_pandas(query)

      totals = pd.json_normalize(df['totals'])
      totals.columns = ['total_' + col for col in totals.columns]
      df = pd.concat([df.drop(columns=['totals']), totals], axis=1)

      source = pd.json_normalize(df['trafficSource'])
      source.columns = ['trafficSource_' + col for col in source.columns]
      df = pd.concat([df.drop(columns=['trafficSource']), source], axis=1)

      device = pd.json_normalize(df['device'])
      device.columns = ['device_' + col for col in device.columns]
      df = pd.concat([df.drop(columns=['device']), device], axis=1)

      network = pd.json_normalize(df['geoNetwork'])
      network.columns = ['geoNetwork_' + col for col in network.columns]
      df = pd.concat([df.drop(columns=['geoNetwork']), network], axis=1)

      custom_dimensions = pd.json_normalize(df.explode('customDimensions')['customDimensions'])
      custom_dimensions.columns = ['customDimensions_' + col for col in custom_dimensions.columns]
      df = pd.concat([df.drop(columns=['customDimensions']), custom_dimensions], axis=1)

      # df.to_csv(f'{table_name}.csv')
      all_data.append(df)

      hits_df = pd.json_normalize(df.explode('hits')['hits'])
      repeated_visit_ids = pd.DataFrame(np.repeat(df['visitId'].values, df['hits'].apply(len)), columns=['visitID'])
      hits_df = pd.concat([repeated_visit_ids, hits_df], axis=1)
      # hits_df.to_csv(f'{table_name}_hits.csv')
      all_hits_data.append(hits_df)

      current_date += timedelta(days=1)

  final_df = pd.concat(all_data, ignore_index=True)
  final_hits_df = pd.concat(all_hits_data, ignore_index=True)

  final_df.to_csv('all_data.csv', index=False)
  final_hits_df.to_csv('all_hits_data.csv', index=False)


# Load Data from Google Drive

In [2]:
# Connect to GDrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
DF_DATA_PATH = '/content/drive/MyDrive/google_analytics_sample/all_data.csv'
HITS_DATA_PATH = '/content/drive/MyDrive/google_analytics_sample/all_hits_data.csv'
def load_final_df(path):
    df = pd.read_csv(path)
    return df
df = load_final_df(DF_DATA_PATH)
#hits_df = load_final_df(HITS_DATA_PATH)

<ipython-input-3-c114907bf6cc>:4: DtypeWarning: Columns (6,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [ ]:
# General Exploration

unique_visits = hits_df['visitID'].unique() # these are not unique, but are unique to each user.

# Given visitId's are not unique to a given date, we need to adapt the data.
# grouped_counts = df.groupby("visitId")['fullVisitorId'].value_counts() # multiple visitID's
# filtered_counts = grouped_counts[grouped_counts > 1]
# print(filtered_counts.to_string())

#print(df[df['visitId'] == 1470120798])
print(hits_df.columns.to_list())

unique_visitors = df['fullVisitorId'].unique()

# Ensuring that we have a truly unique of hits df (based on visitID & visitorID)
# I cannot do this work on Google Colab. Not enough RAM

# hits_df_temp = pd.json_normalize(df.explode('hits')['hits'])
# df['combinedId'] = df['visitId'].astype(str) + '_' + df['fullVisitorId'].astype(str)
# repeated_combined_ids = pd.DataFrame(np.repeat(df['combinedId'].values, df['hits'].apply(len)), columns=['combinedId'])
# unique_hits_df = pd.concat([repeated_combined_ids, hits_df_temp], axis=1)

ecommerce_counts = hits_df.groupby('visitID')['eCommerceAction.action_type'].value_counts().unstack(fill_value=0) # this will work when hits data is unique

len(unique_visitors) #718,161

cols = df.columns
print(cols)
print(ecommerce_counts[0])

# meet w/ group on these, given challenges in linking visitID to visitorID & date
hit_features = [
    "num_product_list_views", #1
    "num_product_detail_views", #2
    "num_add_to_cart", #3
    "num_remove_from_cart", #4
    "checkout", #5
    "purchase", #6
]

In [4]:
from sklearn.preprocessing import LabelEncoder
# TZ in data: GMT-7

df = df.sort_values(by=['fullVisitorId', 'date'])

feature_columns = [
    "fullVisitorId",
    "visitNumber",
    "date",
    "total_hits",
    "total_pageviews",
    "total_screenviews",
    "total_sessionQualityDim",
    "total_timeOnSite",
    "total_totalTransactionRevenue",
    "total_transactions",
    "trafficSource_source",
    "socialEngagementType",
    "channelGrouping",
    "device_browser",
    "device_operatingSystem",
    "device_deviceCategory",
    "geoNetwork_continent",
    "geoNetwork_subContinent",
    "geoNetwork_country",
    "geoNetwork_region",
    "geoNetwork_metro",
    "geoNetwork_city",
]

df = df[feature_columns]

numeric_columns = [
    "visitNumber",
    "total_hits",
    "total_pageviews",
    "total_screenviews",
    "total_sessionQualityDim",
    "total_timeOnSite",
    "total_totalTransactionRevenue",
    "total_transactions"
]

non_numeric_columns = [
    "trafficSource_source",
    "socialEngagementType",
    "channelGrouping",
    "device_browser",
    "device_operatingSystem",
    "device_deviceCategory",
    "geoNetwork_continent",
    "geoNetwork_subContinent",
    "geoNetwork_country",
    "geoNetwork_region",
    "geoNetwork_metro",
    "geoNetwork_city",
]

# Encode non-numeric columns
label_encoders = {}
for col in non_numeric_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

unique_users = df['fullVisitorId'].unique()
unique_dates = df['date'].unique()

# - 2 because fullVisitorId and date are not features
array_shape = (len(unique_users), len(feature_columns) - 2, len(unique_dates))
array_3d = np.zeros(array_shape, dtype=np.float32)

# Create a dictionary to map fullVisitorId and date to index
user_index = {user: i for i, user in enumerate(unique_users)}
date_index = {date: j for j, date in enumerate(unique_dates)}


for row in df.itertuples(index=False):
    user_idx = user_index[row.fullVisitorId]
    date_idx = date_index[row.date]

    # Numeric values
    array_3d[user_idx, 0:len(numeric_columns), date_idx] = [getattr(row, col) for col in numeric_columns]

    # Non-numeric values
    array_3d[user_idx, len(numeric_columns):, date_idx] = [getattr(row, col) for col in non_numeric_columns]

print(array_3d.shape)
print(array_3d)


(718161, 20, 366)
[[[  1.   0.   0. ...   0.   0.   0.]
  [  1.   0.   0. ...   0.   0.   0.]
  [  1.   0.   0. ...   0.   0.   0.]
  ...
  [375.   0.   0. ...   0.   0.   0.]
  [ 93.   0.   0. ...   0.   0.   0.]
  [648.   0.   0. ...   0.   0.   0.]]

 [[  0.   1.   0. ...   0.   0.   0.]
  [  0.  10.   0. ...   0.   0.   0.]
  [  0.   8.   0. ...   0.   0.   0.]
  ...
  [  0. 375.   0. ...   0.   0.   0.]
  [  0.  93.   0. ...   0.   0.   0.]
  [  0. 648.   0. ...   0.   0.   0.]]

 [[  0.   0.   1. ...   0.   0.   0.]
  [  0.   0.  11. ...   0.   0.   0.]
  [  0.   0.   8. ...   0.   0.   0.]
  ...
  [  0.   0. 375. ...   0.   0.   0.]
  [  0.   0.  93. ...   0.   0.   0.]
  [  0.   0. 648. ...   0.   0.   0.]]

 ...

 [[  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  ...
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]
  [  0.   0.   0. ...   0.   0.   0.]]

 [[  0.   0.   0. ...   0.   0. 

## Initial Data Pre-Processing and Feature Extraction

In [ ]:
## code block for preprocessing

## after binding rows we will need to extract the data from the columns with multiple data and put into their own columns

## should code categorical variables into one-encoding

## we need to manipulate and aggregate each user's data for each month for all the manipulated features in our list

## we should take a look at how bad we're missing data and decide for imputation or removal




## shaping the data in the right way for the LSTM model (3 dimensions)

def create_sequences(df, sequence_length):
    sequences = []
    targets = []
    user_ids = df['user_id'].unique()

    for user_id in user_ids:
        user_data = df[df['user_id'] == user_id]
        user_data = user_data.sort_values(by='month')  # Ensure data is sorted by time
        for i in range(len(user_data) - sequence_length):
            seq = user_data.iloc[i:i+sequence_length].drop(columns=['user_id', 'days_until_next_visit']).values # target can change depending on our attrition statistics
            target = user_data.iloc[i+sequence_length]['days_until_next_visit'] # target can change depending on our attrition statistics
            sequences.append(seq)
            targets.append(target)

    return sequences, np.array(targets)

# 12 months
sequence_length = 12

# creating our sequences and targets for putting into X and y objects
data_sequences, targets = create_sequences(df, sequence_length)

# Pad sequences to ensure all have the same length
data_sequences_padded = pad_sequences(data_sequences, maxlen=sequence_length, padding='post', dtype='float32')

# Convert to arrays and saving as X and y
X = np.array(data_sequences_padded)
y = np.array(targets)



## Splitting Data

In [ ]:
## splitting data code block

## need to see distribution of target variable and distribution of months in the data. If unequal need to sample so that the training set has roughly equal targets and months

## may need to manually split the data, make sure we are splitting by user and not by month


## More Data Processing and EDA

In [ ]:
from sklearn.preprocessing import StandardScaler


## EDA - visualizing distributions and correlations


    # histograms for each feature


    # correlations matrix/plot/heatmap






## standardizing features based on the training distributions



# defining lists of indices for the numerical and categorical features and telling it to ignore one-hot encoded variables

# replace with indices for all the numeric columns to be standardized (need to calculate which indices after flattening from 3d to 2d)
numerical_features_indices = [None,None]

# replace with indices for all the one-hot encoded columns to be ignored (need to calculate which indices after flattening from 3d to 2d)
one_hot_encoded_indices = [None,None]




# making a preprocessor to standardize and ignore features (also this is were we can impute data if we'd like)
numerical_transformer = Pipeline(steps=[ # this says to do the below manipulations to each specified column
    ('imputer', SimpleImputer(strategy='median')),  # This is the handling of missing values, placeholder for now
    ('scaler', StandardScaler()) # this standardizes
])

# Combine transformers into a ColumnTransformer
preprocessor = ColumnTransformer( # this takes the pipeline from before and maps which columns to apply the numerical transformations
    transformers=[
        ('num', numerical_transformer, numerical_features_indices),
        ('cat', 'passthrough', one_hot_encoded_indices)
    ]
)



# fitting on 2d reshaped X_train because the scaler won't work on 3 dimensions, then returning back into 3 dimensions
X_train = preprocessor.fit_transform(X_train.reshape(X_train.shape[0]*X_train.shape[1], X_train.shape[-1])).reshape(X_train.shape)
# doing same for test set
X_test = preprocessor.transform(X_test.reshape(X_train.shape[0]*X_train.shape[1], X_test.shape[-1])).reshape(X_test.shape)




## Model Fitting and Architecture

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, TimeDistributed


# may need to turn this into a function so we can use in hyperparameter tuning


## Defining Model

# need to separate the stable features from the dynamic features fix this once we know which features
X_train_dynamic = None
X_train_stable = None


# Input for dynamic features
dynamic_input = Input(shape=(X_train_dynamic.shape[1], X_train_dynamic.shape[2]))
x = LSTM(units=50, return_sequences=True)(dynamic_input) # the parantheses here are telling the function to apply only to the dynamic features


# Input for stable features
stable_input = Input(shape=(X_train_stable.shape[1],))
stable_repeated = Dense(X_train_dynamic.shape[1])(stable_input)
stable_repeated = RepeatVector(X_train_dynamic.shape[1])(stable_repeated)
stable_repeated = TimeDistributed(Dense(50))(stable_repeated)


# Concatenate LSTM output with stable features
x = Concatenate()([x, stable_input])

# TimeDistributed Dense layer to get prediction for each timestep
output = TimeDistributed(Dense(1, activation='sigmoid'))(x)

# Define the model
model = Model(inputs=[dynamic_input, stable_input], outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy', 'precision'])
# should look as precision because we care more about correctly catching attrition over falsely claiming someone will attrit

# Train the model
model.fit([X_train_dynamic, X_train_stable], y_train_scaled, epochs=10, validation_split=0.2)



## Hyperparameter Tuning

In [ ]:
## placeholder

## we need to see how long it takes to fit one epoch, then do some math to see how many sets of hyperparameters we can test

## Evaluation Metrics

In [ ]:
## placeholder